In [1]:
#import pkg_resources
#pkg_resources.require("blosc==1.10.2")
#pkg_resources.require("lz4==3.1.10")
import ROOT
from dask.distributed import Client, LocalCluster


#cluster = LocalCluster(n_workers=4, threads_per_worker=1, processes=True)
#client = Client(threads_per_worker=1, processes=True)
client = Client("172.21.0.2:8786")
RDataFrame = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame
RunGraphs = ROOT.RDF.Experimental.Distributed.RunGraphs

Welcome to JupyROOT 6.26/00


In [2]:
client.dashboard_link

'http://172.21.0.2:8787/status'

In [3]:
# Create a data frame and add two columns: one for the values and one for the weight.
import time
start = time.time()

df = RDataFrame(1000, daskclient=client)

# Set the random seed and define two columns of the dataset with random numbers.
ROOT.gRandom.SetSeed(1)
df_1 = df.Define("gaus", "gRandom->Gaus(10, 1)").Define("exponential", "gRandom->Exp(10)")

# Book an histogram for each column
h_gaus = df_1.Histo1D(("gaus", "Normal distribution", 50, 0, 30), "gaus")
h_exp = df_1.Histo1D(("exponential", "Exponential distribution", 50, 0, 30), "exponential")

# Plot the histograms side by side on a canvas
c = ROOT.TCanvas("distrdf002", "distrdf002", 800, 400)
c.Divide(2, 1)
c.cd(1)
h_gaus.DrawCopy()
c.cd(2)
h_exp.DrawCopy()

# Save the canvas
end = time.time()
print(end-start)


2022-04-06 21:27:54,948 - distributed.protocol.pickle - INFO - Failed to deserialize <memory at 0x7fa28151d700>
Traceback (most recent call last):
  File "/home/break/.local/lib/python3.9/site-packages/distributed/protocol/pickle.py", line 66, in loads
    return pickle.loads(x)
  File "/home/break/Code/root_install/lib/ROOT/_facade.py", line 153, in _importhook
    return _orig_ihook(name, *args, **kwds)
ModuleNotFoundError: No module named 'libROOTPythonizations3_8'
2022-04-06 21:27:54,949 - distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "/home/break/.local/lib/python3.9/site-packages/distributed/protocol/core.py", line 111, in loads
    return msgpack.loads(
  File "msgpack/_unpacker.pyx", line 194, in msgpack._cmsgpack.unpackb
  File "/home/break/.local/lib/python3.9/site-packages/distributed/protocol/core.py", line 103, in _decode_default
    return merge_and_deserialize(
  File "/home/break/.local/lib/python3.9/site-packages

ModuleNotFoundError: No module named 'libROOTPythonizations3_8'

NameError: name 'histoproxies' is not defined

In [5]:
df = RDataFrame("Events", 
                "root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root", 
                daskclient=client)

In [6]:
# For simplicity, select only events with exactly two muons and require opposite charge
df_2mu = df.Filter("nMuon == 2", "Events with exactly two muons")
df_os = df_2mu.Filter("Muon_charge[0] != Muon_charge[1]", "Muons with opposite charge")
 
# Compute invariant mass of the dimuon system
df_mass = df_os.Define("Dimuon_mass", "InvariantMass(Muon_pt, Muon_eta, Muon_phi, Muon_mass)")
 
# Make histogram of dimuon mass spectrum. Note how we can set titles and axis labels in one go.
h = df_mass.Histo1D(("Dimuon_mass", "Dimuon mass;m_{#mu#mu} (GeV);N_{Events}", 30000, 0.25, 300), "Dimuon_mass")


In [7]:
# Request cut-flow report
report = df_mass.Report()

AttributeError: 'Define' object has no attribute 'Report'

In [8]:
# Produce plot
ROOT.gStyle.SetOptStat(0); ROOT.gStyle.SetTextFont(42)
c = ROOT.TCanvas("c", "", 800, 700)
c.SetLogx(); c.SetLogy()
 
h.SetTitle("")
h.GetXaxis().SetTitleSize(0.04)
h.GetYaxis().SetTitleSize(0.04)
h.Draw()
 
label = ROOT.TLatex(); label.SetNDC(True)
label.DrawLatex(0.175, 0.740, "#eta")
label.DrawLatex(0.205, 0.775, "#rho,#omega")
label.DrawLatex(0.270, 0.740, "#phi")
label.DrawLatex(0.400, 0.800, "J/#psi")
label.DrawLatex(0.415, 0.670, "#psi'")
label.DrawLatex(0.485, 0.700, "Y(1,2,3S)")
label.DrawLatex(0.755, 0.680, "Z")
label.SetTextSize(0.040); label.DrawLatex(0.100, 0.920, "#bf{CMS Open Data}")
label.SetTextSize(0.030); label.DrawLatex(0.630, 0.920, "#sqrt{s} = 8 TeV, L_{int} = 11.6 fb^{-1}")
 
c.SaveAs("dimuon_spectrum_distributed.pdf")

Info in <TCanvas::Print>: pdf file dimuon_spectrum_distributed.pdf has been created


In [ ]:
report.Print()

2022-04-06 15:29:28,013 - distributed.nanny - WARNING - Restarting worker
